do pliku `build.gradle(Module)`

```kotlin
dependencies {

    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.2")

    ...
}
```

```kotlin
class MainActivity : ComponentActivity() {
    private val viewModelFactory = object : ViewModelProvider.Factory {
        override fun <T : ViewModel> create(modelClass: Class<T>): T {
            return UserViewModel(UserRepository()) as T
        }
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            FlowOnBasicsExampleTheme {
                UserListScreen(viewModel = viewModel(factory = viewModelFactory))
            }
        }
    }
}

// =================================================================================
// --- Repozytorium ---
// =================================================================================
data class User(val id: Int, val name: String)
class UserRepository {
    // Ta funkcja zwraca ZIMNY Flow.
    // Sama w sobie nie przełącza wątków.
    fun getUsersStream(): Flow<List<User>> = flow {
        delay(1500)
        emit(listOf(
            User(1, "Anna"),
            User(2, "Piotr"),
            User(3, "Zofia")))
    }
}

// =================================================================================
// --- ViewModel ---
// =================================================================================

class UserViewModel(repository: UserRepository) : ViewModel() {

    val uppercasedUserNames: StateFlow<List<String>> =
        repository.getUsersStream()
        .flowOn(Dispatchers.IO)
        .map { users ->
            users.map { it.name.uppercase() }
        }
        .stateIn(
            scope = viewModelScope,
            started = SharingStarted.WhileSubscribed(5000),
            initialValue = emptyList()
        )
}

// =================================================================================
// --- UI (Widok) ---
// =================================================================================

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun UserListScreen(viewModel: UserViewModel) {
    val userNames by viewModel.uppercasedUserNames.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Demo flowOn") }) }) { padding ->
        Column(
            modifier = Modifier.fillMaxSize().padding(padding).padding(16.dp),
            horizontalAlignment = Alignment.CenterHorizontally
        ) {
            Text(
                "Lista Użytkowników (zmienione na wielkie litery)",
                style = MaterialTheme.typography.headlineSmall,
                textAlign = TextAlign.Center
            )
            Spacer(Modifier.height(16.dp))

            if (userNames.isEmpty()) {
                CircularProgressIndicator()
            } else {
                LazyColumn {
                    items(userNames) { name ->
                        Text(name, style = MaterialTheme.typography.bodyLarge, modifier = Modifier.padding(8.dp))
                    }
                }
            }
        }
    }
}
```